In [1]:
import tensorflow as tf
import torch
from accelerate import Accelerator

# CUDA 빌드 여부 확인 (TensorFlow 2.x)
print(f"Built with CUDA: {tf.test.is_built_with_cuda()}")

# CUDA 빌드 여부 확인 (PyTorch)
accelerator = Accelerator()
device = accelerator.device
print(device)

c:\Users\user\anaconda3\envs\local_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Built with CUDA: True
cuda


In [3]:
import unsloth
from unsloth import FastLanguageModel
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer


# 최대 시퀀스 길이를 설정합니다. 내부적으로 RoPE 스케일링을 자동으로 지원합니다!
max_seq_length = 2048  # 공식 튜토리얼은 2048
# 자동 감지를 위해 None을 사용합니다. Tesla T4, V100은 Float16, Ampere+는 Bfloat16을 사용하세요.
dtype = None
# 메모리 사용량을 줄이기 위해 4bit 양자화를 사용합니다. False일 수도 있습니다.
load_in_4bit = True


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit", # Unsloth가 제공하는 최적화된 모델 사용 가능
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # 0보다 큰 어떤 숫자도 선택 가능! 8, 16, 32, 64, 128이 권장됩니다.
    lora_alpha=32,  # LoRA 알파 값을 설정합니다. # 튜토리얼은 16
    lora_dropout=0.05,  # 드롭아웃을 지원합니다. # # Supports any, but = 0 is optimized
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],  # 타겟 모듈을 지정합니다.
    bias="none",  # 바이어스를 지원합니다.
    # True 또는 "unsloth"를 사용하여 매우 긴 컨텍스트에 대해 VRAM을 30% 덜 사용하고, 2배 더 큰 배치 크기를 지원합니다.
    use_gradient_checkpointing="unsloth",
    random_state=123,  # 난수 상태를 설정합니다. # 공식 : 3407
    use_rslora=False,  # 순위 안정화 LoRA를 지원합니다.
    loftq_config=None,  # LoftQ를 지원합니다.
)


==((====))==  Unsloth 2025.5.4: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 3050. Num GPUs = 1. Max memory: 6.0 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.7.0+cu118. CUDA: 8.6. CUDA Toolkit: 11.8. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.5.4 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [5]:

# 테스트를 위한 프롬프트 준비
prompt = """리눅스를 잘 다루기 위해서는 어떤 것들을 알아야 할까요 ?""" # 테스트하고 싶은 질문으로 변경 가능

# Unsloth에서 제공하는 프롬프트 형식을 사용할 수도 있습니다.
# 예시: Alpaca 형식 (모델이 Alpaca 형식으로 미세 조정되었다고 가정)
# alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

# ### Instruction:
# {}

# ### Response:
# {}"""
# inputs = tokenizer(
# [
#     alpaca_prompt.format(
#         "당신은 누구인가요?", # instruction
#         "", # output - 생성 부분이므로 비워둡니다.
#     )
# ], return_tensors = "pt").to("cuda")


# 간단한 프롬프트를 직접 토큰화
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

# 모델을 사용하여 응답 생성
# FastLanguageModel은 내부적으로 model.generate를 최적화할 수 있습니다.
# generate_params = {"max_new_tokens": 50, "use_cache": True} # 필요한 경우 생성 파라미터 조정
# outputs = model.generate(**inputs, **generate_params)

# 더 간단한 방식
outputs = model.generate(**inputs, max_new_tokens=50)


# 생성된 응답 디코딩
response_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

print("모델 입력:")
print(prompt)
print("\n모델 응답:")
print(response_text)

# 메모리 정리 (필요한 경우)
# import gc
# del inputs, outputs
# gc.collect()
# torch.cuda.empty_cache()
# </VSCode.Cell>

모델 입력:
리눅스를 잘 다루기 위해서는 어떤 것들을 알아야 할까요 ?

모델 응답:
리눅스를 잘 다루기 위해서는 어떤 것들을 알아야 할까요? 
1.  리눅스 시스템의 기본적인 workings을 이해해야 합니다.
2.  리눅스에 대한 kiến지의를 쌓아야 합니다.
3.  리눅스에 대한 지식과 경험


In [ ]:
""